### Analyze racurses tflite model on real videos

In [448]:
import tensorflow as tf

import numpy as np
from PIL import Image

import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1500)
pd.set_option('display.max_colwidth', 1000)
# pd.set_option('display.max_colheight', 1000)
pd.set_option('display.max_rows', 500)

import glob
from IPython.display import Image as iimage

In [6]:
racurs_model_path = 'ourmodels/bundled-tflite-racurses_2020219191728-2020-02-20T12_52_18.283Z/model.tflite'
racurs_label_path = 'ourmodels/bundled-tflite-racurses_2020219191728-2020-02-20T12_52_18.283Z/dict.txt'

In [7]:
def load_labels(filename):
    with open(filename, 'r') as f:
        return [line.strip() for line in f.readlines()]
labels = load_labels(racurs_label_path)

In [10]:
interpreter = tf.lite.Interpreter(racurs_model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [11]:
input_ddetails

[{'name': 'image',
  'index': 3,
  'shape': array([  1, 224, 224,   3], dtype=int32),
  'dtype': numpy.uint8,
  'quantization': (0.007874015718698502, 128)}]

In [12]:
output_details

[{'name': 'scores',
  'index': 172,
  'shape': array([1, 6], dtype=int32),
  'dtype': numpy.uint8,
  'quantization': (0.00390625, 0)}]

In [38]:
img_path = 'images_from_video/videos_ok_53b_d6a_53bd6a46-5540-4428-a879-4d4060f15ad7/1.jpg' # 3/4 back
img_path = 'images_from_video/videos_ok_53b_d6a_53bd6a46-5540-4428-a879-4d4060f15ad7/500.jpg' # 3/4 front

In [49]:
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]
# img_path = '/var/tmp/tmpjcmgv9u0.jpg'
img = Image.open(img_path).resize((width, height))
input_data = np.expand_dims(img, axis = 0)
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
scores = interpreter.get_tensor(output_details[0]['index'])

In [50]:
scores

array([[  8, 233,   6,  23,  29,  87]], dtype=uint8)

In [40]:
LABEL_2_READABLE_RACURS = {'0': "auto_view_3_4_back_0",
                           '1': "auto_view_3_4_front_1",
                           '2': "auto_view_back_2",
                           '7': "auto_view_front_7",
                           '18': "auto_view_side_18",
                           "100": "other_views_100"}

In [41]:
labels

['18', '1', '2', '100', '0', '7']

In [705]:
# iimage(img_path)

### Get racurs from image path

In [71]:
interpreter = tf.lite.Interpreter(racurs_model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
def get_racurs_tf_lite_result(img_path):
    height = input_details[0]['shape'][1]
    width = input_details[0]['shape'][2]
    img = Image.open(img_path).resize((width, height))
    input_data = np.expand_dims(img, axis = 0)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    scores = interpreter.get_tensor(output_details[0]['index']).squeeze()
#     print(scores)
    max_index = np.argmax(scores)
    final_score = scores[max_index]
    racurs = LABEL_2_READABLE_RACURS[labels[max_index]]
    return racurs, final_score

In [72]:
get_racurs_tf_lite_result(img_path)

('auto_view_side_18', 185)

In [111]:
img_path = 'images_from_video/videos_ok_53b_d6a_53bd6a46-5540-4428-a879-4d4060f15ad7/1.jpg' # 3/4 back
img_path = 'images_from_video/videos_ok_53b_d6a_53bd6a46-5540-4428-a879-4d4060f15ad7/1000.jpg' # 3/4 back
img_path = 'images_from_video/videos_ok_53b_d6a_53bd6a46-5540-4428-a879-4d4060f15ad7/200.jpg' # 3/4 back
print(get_racurs_tf_lite_result(img_path))
# iimage(img_path)

('auto_view_side_18', 185)


### Analyze video

In [173]:
df_bb_classifier = pd.read_csv('classification_results_v2.tsv', sep = '\t')


In [179]:
df_bb_classifier.head()

,url,panorama_status,image_path,classifier_result,total_analyzed_frames,big_vehicle_middle_screen_found,no_big_vehicle_middle_screen_found,car_found_but_bounding_box_unsafe_margin,bounding_boxes_outside_image,bad_frames,really_bad_frames,share_no_big_vehicle_middle_screen_found,share_car_found_but_bounding_box_unsafe_margin,share_bad_frames,share_really_bad_frames
0,https://autoru-panorama-internal.s3.mds.yandex.net/upload/d98/a9f/d98a9f97-7491-4bed-8b32-acb49de28be8,failed,images_from_video/videos_failed_d98_a9f_d98a9f97-7491-4bed-8b32-acb49de28be8,GOOD,147,144,0,0,0,3,3,0.020408,0.00000,0.020408,0.020408
1,https://autoru-panorama-internal.s3.mds.yandex.net/upload/e86/89d/e8689d6f-1bf9-4b83-a592-82ecf4956e83,completed,images_from_video/videos_ok_e86_89d_e8689d6f-1bf9-4b83-a592-82ecf4956e83,GOOD,80,80,0,7,2,7,2,0.000000,0.08750,0.087500,0.025000
2,https://autoru-panorama-internal.s3.mds.yandex.net/upload/287/b38/287b380e-e940-44d1-bf0d-e4e1401dd015,completed,images_from_video/videos_ok_287_b38_287b380e-e940-44d1-bf0d-e4e1401dd015,GOOD,157,154,0,0,0,3,3,0.019108,0.00000,0.019108,0.019108
3,https://autoru-panorama-internal.s3.mds.yandex.net/upload/53b/d6a/53bd6a46-5540-4428-a879-4d4060f15ad7,completed,images_from_video/videos_ok_53b_d6a_53bd6a46-5540-4428-a879-4d4060f15ad7,BAD,89,82,0,37,12,44,19,0.078652,0.45122,0.494382,0.213483
4,https://autoru-panorama-internal.s3.mds.yandex.net/upload/dad/56f/dad56ff5-34be-4a46-afb7-22962a9b37df,completed,images_from_video/videos_ok_dad_56f_dad56ff5-34be-4a46-afb7-22962a9b37df,GOOD,144,143,0,0,0,1,1,0.006944,0.00000,0.006944,0.006944


In [178]:
GOOD = 'GOOD'
# COMPLETED = 'completed'
# FAILED = 'failed'
list(df_bb_classifier.image_path)[0:2]

['images_from_video/videos_failed_d98_a9f_d98a9f97-7491-4bed-8b32-acb49de28be8',
 'images_from_video/videos_ok_e86_89d_e8689d6f-1bf9-4b83-a592-82ecf4956e83']

In [146]:
from functools import cmp_to_key

def sort_by_filenumber(x, y):
    name1 = int(x.split('/')[-1].split('.jpg')[0])
    name2 = int(y.split('/')[-1].split('.jpg')[0])
#     print(name1)
#     print(name2)
    return -1 if name1 < name2 else 1
    
    
# sort_by_filenumber('images_from_video/videos_ok_53b_d6a_53bd6a46-5540-4428-a879-4d4060f15ad7/1.jpg', 'images_from_video/videos_ok_53b_d6a_53bd6a46-5540-4428-a879-4d4060f15ad7/1000.jpg')    
folder_path = 'images_from_video/videos_failed_00e_725_00e725e3-afad-4b46-be56-46664c51d8c4/'
all_files = glob.glob(folder_path + "*.jpg")
    
all_files.sort(key=cmp_to_key(sort_by_filenumber))

In [216]:
from collections import defaultdict
# folder_path = 'images_from_video/videos_failed_00e_725_00e725e3-afad-4b46-be56-46664c51d8c4'
def process_video_though_racurses(folder_path):
    all_files = glob.glob(folder_path + "/*.jpg")
    all_files.sort(key=cmp_to_key(sort_by_filenumber))
    list_counts = []
    racurses_dict = defaultdict(int)
    i = 0
    for file in all_files:
        # process 2 frames per second
        i += 1
        if i%12 != 1:
            continue
        racurs, score = get_racurs_tf_lite_result(file)
        racurses_dict[racurs] += 1
        if len(list_counts) == 0:
            add_dict = {}
            add_dict[racurs] = 1
            list_counts.append(add_dict)
        else:
            if racurs in list_counts[-1]:
                list_counts[-1][racurs] += 1
            else:
                add_dict = {}
                add_dict[racurs] = 1
                list_counts.append(add_dict)
#         print(list_counts)
#         print(racurses_dict)
    return dict(racurses_dict), list_counts

In [170]:
racurses_dict, list_counts = process_video_though_racurses(folder_path = folder_path)

In [171]:
racurses_dict

{'auto_view_front_7': 42,
 'auto_view_side_18': 82,
 'other_views_100': 3,
 'auto_view_back_2': 2,
 'auto_view_3_4_back_0': 1,
 'auto_view_3_4_front_1': 1}

In [182]:
all_keys = list(racurses_dict.keys())
# if len(all_keys) == 6 
if 'other_views_100' in all_keys:
    print('ok')
 

ok


In [222]:
def are_all_racurses_present(racurses_dict):
    all_keys = list(racurses_dict.keys())
    total_keys = len(all_keys)
    if total_keys == 6 or (total_keys == 5 and 'other_views_100' not in all_keys):
        return True
    else:
        return False
   

In [172]:
list_counts

[{'auto_view_front_7': 39},
 {'auto_view_side_18': 28},
 {'other_views_100': 3},
 {'auto_view_side_18': 6},
 {'auto_view_back_2': 2},
 {'auto_view_side_18': 32},
 {'auto_view_3_4_back_0': 1},
 {'auto_view_side_18': 12},
 {'auto_view_3_4_front_1': 1},
 {'auto_view_side_18': 4},
 {'auto_view_front_7': 3}]

In [223]:
all_dicts = []
def process_all_videos(folder_paths):
    for path in folder_paths:
        print(f"Processing {path}")
        racurses_dict, list_counts = process_video_though_racurses(path)
        res = {}
        res['image_path'] = path
        res['racurses_dict'] = racurses_dict
        res['list_racurses'] = list_counts
        res['all_racurses_present'] = are_all_racurses_present(racurses_dict)
        all_dicts.append(res)
    return pd.DataFrame(all_dicts)

In [226]:
# df_res_racurses1 = process_all_videos(list(df_bb_classifier.image_path)[0:2])
df_res_racurses = process_all_videos(list(df_bb_classifier.image_path))

Processing images_from_video/videos_failed_d98_a9f_d98a9f97-7491-4bed-8b32-acb49de28be8
Processing images_from_video/videos_ok_e86_89d_e8689d6f-1bf9-4b83-a592-82ecf4956e83
Processing images_from_video/videos_ok_287_b38_287b380e-e940-44d1-bf0d-e4e1401dd015
Processing images_from_video/videos_ok_53b_d6a_53bd6a46-5540-4428-a879-4d4060f15ad7
Processing images_from_video/videos_ok_dad_56f_dad56ff5-34be-4a46-afb7-22962a9b37df
Processing images_from_video/videos_ok_da4_dcd_da4dcd26-08af-4ac9-b233-775f34c89bce
Processing images_from_video/videos_ok_e13_766_e1376683-cec9-440b-b5ef-ce831686f60d
Processing images_from_video/videos_ok_03d_04a_03d04ae5-1954-426c-b6ed-2c997f3ddcca
Processing images_from_video/videos_ok_89b_d79_89bd793d-2069-421d-beff-7a93c8874a95
Processing images_from_video/videos_ok_31e_57f_31e57f3e-a6fb-475f-a204-0fd9b8250be4
Processing images_from_video/videos_ok_46a_061_46a061a5-b242-49b1-a41e-65f13b003784
Processing images_from_video/videos_ok_a4f_99d_a4f99db1-7605-449f-9b9f-c

Processing images_from_video/videos_ok_6b1_300_6b130009-2529-453b-a44c-6c7ff840e41c
Processing images_from_video/videos_ok_c6e_199_c6e199e7-061d-4641-ad32-2b3b9889fb7c
Processing images_from_video/videos_ok_f1b_f0a_f1bf0a37-8193-44ed-9605-da1bd29fd043
Processing images_from_video/videos_ok_591_830_59183004-1a31-4b29-9122-74b201574e76
Processing images_from_video/videos_failed_ebb_b2f_ebbb2f60-3561-4fb8-a279-cb22550d743d
Processing images_from_video/videos_ok_08e_183_08e183ad-dc6c-4bff-bb03-bc0f72ea1a3c
Processing images_from_video/videos_ok_e4a_ba3_e4aba3a6-46a8-46c3-be30-12fc9a5297e0
Processing images_from_video/videos_failed_00e_725_00e725e3-afad-4b46-be56-46664c51d8c4
Processing images_from_video/videos_ok_59e_aa7_59eaa7fc-c014-40de-a81d-fbb2d83570dc
Processing images_from_video/videos_ok_56b_888_56b8885f-30d4-4975-9bf0-4505ef2e2576
Processing images_from_video/videos_ok_a48_59c_a4859c48-00e9-41f6-b0d1-20ab03648fa4
Processing images_from_video/videos_failed_9b2_c87_9b2c87ff-ab19-400

Processing images_from_video/videos_ok_f23_45b_f2345b1c-6bba-4c54-9f59-e21114ab9df4
Processing images_from_video/videos_ok_228_82e_22882ea0-87fb-42a4-994b-07c8d7ed1e1a
Processing images_from_video/videos_failed_fcb_c69_fcbc6954-0d90-4d9e-9672-12055d16cfdb
Processing images_from_video/videos_failed_ab3_f36_ab3f3671-212b-432e-a24b-a0c20ac2ea12
Processing images_from_video/videos_ok_362_974_362974b0-5e49-4eea-b780-ce13afb6f05c
Processing images_from_video/videos_ok_efc_cd1_efccd157-adde-4cb2-aa33-5f855ff3723e
Processing images_from_video/videos_ok_98e_51a_98e51a65-32e2-492c-a32c-f03431f3596e
Processing images_from_video/videos_ok_8f4_207_8f4207d1-6593-47fc-aad6-d15860f49bb5
Processing images_from_video/videos_ok_f80_579_f8057937-7838-4d5c-b6c1-1034c62aef00
Processing images_from_video/videos_ok_945_c16_945c16e5-fe42-4446-87ce-c6ee2356fee5
Processing images_from_video/videos_ok_952_6bf_9526bfd4-8222-42a2-9ff3-6552a7156f94
Processing images_from_video/videos_ok_4a9_9a0_4a99a0cf-099c-4575-ae

Processing images_from_video/videos_failed_612_b58_612b589e-f752-441e-b8ef-b3c628f734cd
Processing images_from_video/videos_ok_cb0_067_cb006707-faa1-4c99-abb4-0721c730c13f
Processing images_from_video/videos_failed_be6_cce_be6ccecb-b0c8-4e63-b02b-43e5317e4fc9
Processing images_from_video/videos_ok_afe_88c_afe88ce2-ed96-4415-b0d8-297096ba957b
Processing images_from_video/videos_failed_458_fd1_458fd1b7-6881-420a-bdee-a82650bc0f04
Processing images_from_video/videos_ok_36f_9be_36f9bec6-6e95-4c83-a1ac-28cd6a97e1b9
Processing images_from_video/videos_ok_d48_130_d48130d1-db2b-41e7-92bb-d7b333030b1a
Processing images_from_video/videos_ok_5d1_3cb_5d13cb4f-387d-4784-8f30-a81dbcd8287b
Processing images_from_video/videos_failed_6e9_354_6e935483-52d0-4c1f-9079-8876fe77b503
Processing images_from_video/videos_failed_2c4_234_2c42345d-c827-432d-80c1-aa7a3b254133
Processing images_from_video/videos_ok_33d_1bc_33d1bcc1-4407-4d65-a5a9-0da472b56e75
Processing images_from_video/videos_ok_561_615_561615c5-

In [228]:
df_res_racurses.sample(5)

,image_path,racurses_dict,list_racurses,all_racurses_present
5,images_from_video/videos_ok_53b_d6a_53bd6a46-5540-4428-a879-4d4060f15ad7,"{'auto_view_3_4_back_0': 18, 'auto_view_side_18': 28, 'auto_view_3_4_front_1': 22, 'auto_view_front_7': 10, 'auto_view_back_2': 11}","[{'auto_view_3_4_back_0': 3}, {'auto_view_side_18': 16}, {'auto_view_3_4_front_1': 10}, {'auto_view_front_7': 10}, {'auto_view_3_4_front_1': 12}, {'auto_view_3_4_back_0': 2}, {'auto_view_side_18': 12}, {'auto_view_3_4_back_0': 9}, {'auto_view_back_2': 11}, {'auto_view_3_4_back_0': 4}]",True
161,images_from_video/videos_failed_64c_61a_64c61abc-29b0-4665-b308-9b60063c5ab2,"{'auto_view_front_7': 18, 'auto_view_side_18': 72, 'auto_view_3_4_front_1': 3, 'auto_view_3_4_back_0': 5, 'auto_view_back_2': 4, 'other_views_100': 3}","[{'auto_view_front_7': 1}, {'auto_view_side_18': 1}, {'auto_view_3_4_front_1': 1}, {'auto_view_front_7': 17}, {'auto_view_3_4_back_0': 1}, {'auto_view_side_18': 1}, {'auto_view_3_4_back_0': 1}, {'auto_view_side_18': 21}, {'auto_view_3_4_back_0': 1}, {'auto_view_side_18': 1}, {'auto_view_back_2': 1}, {'auto_view_side_18': 28}, {'auto_view_back_2': 1}, {'auto_view_side_18': 3}, {'other_views_100': 3}, {'auto_view_back_2': 2}, {'auto_view_side_18': 9}, {'auto_view_3_4_back_0': 2}, {'auto_view_side_18': 8}, {'auto_view_3_4_front_1': 2}]",True
256,images_from_video/videos_failed_019_f53_019f5330-c9f5-493f-8a29-5f33054e523a,"{'auto_view_front_7': 32, 'auto_view_3_4_front_1': 32, 'auto_view_side_18': 54, 'auto_view_3_4_back_0': 28, 'auto_view_back_2': 18}","[{'auto_view_front_7': 14}, {'auto_view_3_4_front_1': 17}, {'auto_view_side_18': 25}, {'auto_view_3_4_back_0': 1}, {'auto_view_side_18': 1}, {'auto_view_3_4_back_0': 1}, {'auto_view_side_18': 1}, {'auto_view_3_4_back_0': 12}, {'auto_view_back_2': 18}, {'auto_view_3_4_back_0': 14}, {'auto_view_side_18': 27}, {'auto_view_3_4_front_1': 15}, {'auto_view_front_7': 18}]",True
77,images_from_video/videos_ok_15f_4eb_15f4eb71-447b-4356-b817-de444c2ee98a,"{'auto_view_3_4_front_1': 49, 'auto_view_side_18': 73, 'auto_view_3_4_back_0': 13, 'auto_view_back_2': 22, 'other_views_100': 3, 'auto_view_front_7': 36}","[{'auto_view_3_4_front_1': 3}, {'auto_view_side_18': 1}, {'auto_view_3_4_front_1': 8}, {'auto_view_side_18': 1}, {'auto_view_3_4_front_1': 5}, {'auto_view_side_18': 19}, {'auto_view_3_4_back_0': 2}, {'auto_view_side_18': 1}, {'auto_view_3_4_back_0': 2}, {'auto_view_side_18': 1}, {'auto_view_3_4_back_0': 1}, {'auto_view_side_18': 1}, {'auto_view_3_4_back_0': 1}, {'auto_view_3_4_front_1': 14}, {'auto_view_back_2': 17}, {'auto_view_3_4_front_1': 1}, {'auto_view_back_2': 1}, {'auto_view_3_4_front_1': 7}, {'auto_view_3_4_back_0': 4}, {'auto_view_3_4_front_1': 1}, {'auto_view_3_4_back_0': 2}, {'other_views_100': 1}, {'auto_view_3_4_back_0': 1}, {'auto_view_side_18': 2}, {'auto_view_3_4_front_1': 1}, {'auto_view_side_18': 1}, {'other_views_100': 1}, {'auto_view_side_18': 1}, {'other_views_100': 1}, {'auto_view_side_18': 32}, {'auto_view_3_4_front_1': 1}, {'auto_view_front_7': 31}, {'auto_view_back_2': 1}, {'auto_view_front_7': 2}, {'auto_view_back_2': 1}, {'auto_view_front_7': 2}, {'auto_...",True
58,images_from_video/videos_failed_af9_ee7_af9ee745-2571-414e-ae9f-5cde82e9dfda,"{'auto_view_3_4_front_1': 23, 'auto_view_side_18': 27, 'auto_view_3_4_back_0': 23, 'auto_view_front_7': 9, 'auto_view_back_2': 7}","[{'auto_view_3_4_front_1': 13}, {'auto_view_side_18': 1}, {'auto_view_3_4_front_1': 4}, {'auto_view_side_18': 14}, {'auto_view_3_4_front_1': 1}, {'auto_view_side_18': 1}, {'auto_view_3_4_front_1': 2}, {'auto_view_3_4_back_0': 2}, {'auto_view_3_4_front_1': 2}, {'auto_view_3_4_back_0': 3}, {'auto_view_3_4_front_1': 1}, {'auto_view_front_7': 7}, {'auto_view_back_2': 1}, {'auto_view_front_7': 2}, {'auto_view_back_2': 6}, {'auto_view_3_4_back_0': 17}, {'auto_view_side_18': 1}, {'auto_view_3_4_back_0': 1}, {'auto_view_side_18': 10}]",True


In [227]:
!curl --data "text=Finished classifying panorama videos with racurses model tflite @VladimirGorovoy&parse_mode=Markdown" https://api.telegram.org/bot345259261:AAGBS5bxlxTe8I9cp_3mCpP-tghLHAuAOkw/sendMessage?chat_id=-233546442

{"ok":true,"result":{"message_id":12990,"from":{"id":345259261,"is_bot":true,"first_name":"RealtyCallCenterBot","username":"RealtyCallCenterBot"},"chat":{"id":-233546442,"title":"BotTests","type":"group","all_members_are_administrators":true},"date":1586261669,"text":"Finished classifying panorama videos with racurses model tflite @VladimirGorovoy","entities":[{"offset":64,"length":16,"type":"mention"}]}}

In [194]:
type(df_res_racurses)

pandas.core.frame.DataFrame

In [198]:
df_res_racurses.sample(5)

,image_path,racurses_dict,list_racurses,all_racurses_present
313,images_from_video/videos_failed_e63_4de_e634deb8-b54a-43b6-b36a-05ae8211808a,"{'auto_view_front_7': 42, 'auto_view_side_18': 82, 'other_views_100': 3, 'auto_view_back_2': 2, 'auto_view_3_4_back_0': 1, 'auto_view_3_4_front_1': 1}","[{'auto_view_front_7': 39}, {'auto_view_side_18': 28}, {'other_views_100': 3}, {'auto_view_side_18': 6}, {'auto_view_back_2': 2}, {'auto_view_side_18': 32}, {'auto_view_3_4_back_0': 1}, {'auto_view_side_18': 12}, {'auto_view_3_4_front_1': 1}, {'auto_view_side_18': 4}, {'auto_view_front_7': 3}]",True
226,images_from_video/videos_ok_1c0_7ec_1c07ec0d-3b9d-4c31-9b46-abb7bffb51d7,"{'auto_view_front_7': 42, 'auto_view_side_18': 82, 'other_views_100': 3, 'auto_view_back_2': 2, 'auto_view_3_4_back_0': 1, 'auto_view_3_4_front_1': 1}","[{'auto_view_front_7': 39}, {'auto_view_side_18': 28}, {'other_views_100': 3}, {'auto_view_side_18': 6}, {'auto_view_back_2': 2}, {'auto_view_side_18': 32}, {'auto_view_3_4_back_0': 1}, {'auto_view_side_18': 12}, {'auto_view_3_4_front_1': 1}, {'auto_view_side_18': 4}, {'auto_view_front_7': 3}]",True
9,images_from_video/videos_ok_03d_04a_03d04ae5-1954-426c-b6ed-2c997f3ddcca,"{'auto_view_front_7': 42, 'auto_view_side_18': 82, 'other_views_100': 3, 'auto_view_back_2': 2, 'auto_view_3_4_back_0': 1, 'auto_view_3_4_front_1': 1}","[{'auto_view_front_7': 39}, {'auto_view_side_18': 28}, {'other_views_100': 3}, {'auto_view_side_18': 6}, {'auto_view_back_2': 2}, {'auto_view_side_18': 32}, {'auto_view_3_4_back_0': 1}, {'auto_view_side_18': 12}, {'auto_view_3_4_front_1': 1}, {'auto_view_side_18': 4}, {'auto_view_front_7': 3}]",True
246,images_from_video/videos_ok_889_af7_889af702-ad32-43d8-a21a-7ca48d8dfef3,"{'auto_view_front_7': 42, 'auto_view_side_18': 82, 'other_views_100': 3, 'auto_view_back_2': 2, 'auto_view_3_4_back_0': 1, 'auto_view_3_4_front_1': 1}","[{'auto_view_front_7': 39}, {'auto_view_side_18': 28}, {'other_views_100': 3}, {'auto_view_side_18': 6}, {'auto_view_back_2': 2}, {'auto_view_side_18': 32}, {'auto_view_3_4_back_0': 1}, {'auto_view_side_18': 12}, {'auto_view_3_4_front_1': 1}, {'auto_view_side_18': 4}, {'auto_view_front_7': 3}]",True
211,images_from_video/videos_ok_f80_579_f8057937-7838-4d5c-b6c1-1034c62aef00,"{'auto_view_front_7': 42, 'auto_view_side_18': 82, 'other_views_100': 3, 'auto_view_back_2': 2, 'auto_view_3_4_back_0': 1, 'auto_view_3_4_front_1': 1}","[{'auto_view_front_7': 39}, {'auto_view_side_18': 28}, {'other_views_100': 3}, {'auto_view_side_18': 6}, {'auto_view_back_2': 2}, {'auto_view_side_18': 32}, {'auto_view_3_4_back_0': 1}, {'auto_view_side_18': 12}, {'auto_view_3_4_front_1': 1}, {'auto_view_side_18': 4}, {'auto_view_front_7': 3}]",True


In [229]:
df_final = df_bb_classifier.merge(df_res_racurses)

In [230]:
df_final.sample(5)

,url,panorama_status,image_path,classifier_result,total_analyzed_frames,big_vehicle_middle_screen_found,no_big_vehicle_middle_screen_found,car_found_but_bounding_box_unsafe_margin,bounding_boxes_outside_image,bad_frames,really_bad_frames,share_no_big_vehicle_middle_screen_found,share_car_found_but_bounding_box_unsafe_margin,share_bad_frames,share_really_bad_frames,racurses_dict,list_racurses,all_racurses_present
279,https://autoru-panorama-internal.s3.mds.yandex.net/upload/76e/dc9/76edc944-4c2b-4b0b-8847-63d72cfee674,failed,images_from_video/videos_failed_76e_dc9_76edc944-4c2b-4b0b-8847-63d72cfee674,GOOD,176,172,0,0,0,4,4,0.022727,0.000000,0.022727,0.022727,"{'auto_view_front_7': 55, 'auto_view_side_18': 95, 'auto_view_3_4_front_1': 6, 'auto_view_3_4_back_0': 11, 'auto_view_back_2': 9}","[{'auto_view_front_7': 17}, {'auto_view_side_18': 2}, {'auto_view_3_4_front_1': 1}, {'auto_view_side_18': 1}, {'auto_view_3_4_front_1': 4}, {'auto_view_side_18': 6}, {'auto_view_3_4_front_1': 1}, {'auto_view_side_18': 32}, {'auto_view_3_4_back_0': 9}, {'auto_view_side_18': 1}, {'auto_view_3_4_back_0': 1}, {'auto_view_back_2': 2}, {'auto_view_front_7': 7}, {'auto_view_back_2': 2}, {'auto_view_side_18': 1}, {'auto_view_back_2': 1}, {'auto_view_side_18': 1}, {'auto_view_back_2': 1}, {'auto_view_side_18': 1}, {'auto_view_back_2': 1}, {'auto_view_side_18': 2}, {'auto_view_back_2': 1}, {'auto_view_front_7': 1}, {'auto_view_back_2': 1}, {'auto_view_side_18': 2}, {'auto_view_3_4_back_0': 1}, {'auto_view_side_18': 44}, {'auto_view_front_7': 3}, {'auto_view_side_18': 2}, {'auto_view_front_7': 27}]",True
163,https://autoru-panorama-internal.s3.mds.yandex.net/upload/598/223/5982233c-b414-4968-93ab-226aff952573,completed,images_from_video/videos_ok_598_223_5982233c-b414-4968-93ab-226aff952573,BAD,163,162,0,94,58,95,59,0.006135,0.580247,0.582822,0.361963,"{'auto_view_front_7': 23, 'auto_view_3_4_front_1': 7, 'auto_view_3_4_back_0': 47, 'auto_view_side_18': 67, 'auto_view_back_2': 19}","[{'auto_view_front_7': 20}, {'auto_view_3_4_front_1': 7}, {'auto_view_3_4_back_0': 1}, {'auto_view_side_18': 32}, {'auto_view_3_4_back_0': 22}, {'auto_view_back_2': 10}, {'auto_view_front_7': 1}, {'auto_view_back_2': 1}, {'auto_view_front_7': 1}, {'auto_view_back_2': 5}, {'auto_view_front_7': 1}, {'auto_view_back_2': 2}, {'auto_view_3_4_back_0': 1}, {'auto_view_back_2': 1}, {'auto_view_3_4_back_0': 23}, {'auto_view_side_18': 35}]",True
77,https://autoru-panorama-internal.s3.mds.yandex.net/upload/15f/4eb/15f4eb71-447b-4356-b817-de444c2ee98a,completed,images_from_video/videos_ok_15f_4eb_15f4eb71-447b-4356-b817-de444c2ee98a,GOOD,196,196,0,30,3,30,3,0.000000,0.153061,0.153061,0.015306,"{'auto_view_3_4_front_1': 49, 'auto_view_side_18': 73, 'auto_view_3_4_back_0': 13, 'auto_view_back_2': 22, 'other_views_100': 3, 'auto_view_front_7': 36}","[{'auto_view_3_4_front_1': 3}, {'auto_view_side_18': 1}, {'auto_view_3_4_front_1': 8}, {'auto_view_side_18': 1}, {'auto_view_3_4_front_1': 5}, {'auto_view_side_18': 19}, {'auto_view_3_4_back_0': 2}, {'auto_view_side_18': 1}, {'auto_view_3_4_back_0': 2}, {'auto_view_side_18': 1}, {'auto_view_3_4_back_0': 1}, {'auto_view_side_18': 1}, {'auto_view_3_4_back_0': 1}, {'auto_view_3_4_front_1': 14}, {'auto_view_back_2': 17}, {'auto_view_3_4_front_1': 1}, {'auto_view_back_2': 1}, {'auto_view_3_4_front_1': 7}, {'auto_view_3_4_back_0': 4}, {'auto_view_3_4_front_1': 1}, {'auto_view_3_4_back_0': 2}, {'other_views_100': 1}, {'auto_view_3_4_back_0': 1}, {'auto_view_side_18': 2}, {'auto_view_3_4_front_1': 1}, {'auto_view_side_18': 1}, {'other_views_100': 1}, {'auto_view_side_18': 1}, {'other_views_100': 1}, {'auto_view_side_18': 32}, {'auto_view_3_4_front_1': 1}, {'auto_view_front_7': 31}, {'auto_view_back_2': 1}, {'auto_view_front_7': 2}, {'auto_view_back_2': 1}, {'auto_view_front_7': 2}, {'auto_...",True
85,https://autoru-panorama-internal.s3.mds.yandex.net/upload/986/879/986879c4-584d-44d6-89e5-1aa0f37a5983,failed,images_from_video/videos_failed_986_879_9868

In [231]:
df_final.to_csv('bb_racurs_classifier_result.tsv', sep = '\t', index = False)

In [242]:
list(df_final)
cols = [
 'panorama_status',
 'classifier_result',
 'racurses_dict',
 'image_path',
 'list_racurses']
# df_final[]

In [243]:
s = df_final[cols].sample(30)
s

,panorama_status,classifier_result,racurses_dict,image_path,list_racurses
302,completed,GOOD,"{'auto_view_3_4_front_1': 102, 'auto_view_side_18': 69, 'auto_view_3_4_back_0': 57, 'auto_view_back_2': 33, 'auto_view_front_7': 36}",images_from_video/videos_ok_afe_88c_afe88ce2-ed96-4415-b0d8-297096ba957b,"[{'auto_view_3_4_front_1': 44}, {'auto_view_side_18': 43}, {'auto_view_3_4_back_0': 2}, {'auto_view_side_18': 1}, {'auto_view_3_4_back_0': 4}, {'auto_view_3_4_front_1': 3}, {'auto_view_3_4_back_0': 15}, {'auto_view_back_2': 30}, {'auto_view_front_7': 1}, {'auto_view_back_2': 1}, {'auto_view_front_7': 1}, {'auto_view_back_2': 2}, {'auto_view_front_7': 3}, {'auto_view_3_4_back_0': 29}, {'auto_view_side_18': 1}, {'auto_view_3_4_back_0': 2}, {'auto_view_side_18': 1}, {'auto_view_3_4_back_0': 5}, {'auto_view_side_18': 23}, {'auto_view_3_4_front_1': 40}, {'auto_view_front_7': 31}, {'auto_view_3_4_front_1': 15}]"
134,completed,GOOD,"{'auto_view_back_2': 32, 'auto_view_side_18': 22, 'other_views_100': 109, 'auto_view_3_4_back_0': 5, 'auto_view_front_7': 48, 'auto_view_3_4_front_1': 10}",images_from_video/videos_ok_d1f_b8a_d1fb8aee-c342-488c-8c6d-318c9017c047,"[{'auto_view_back_2': 24}, {'auto_view_side_18': 1}, {'auto_view_back_2': 4}, {'auto_view_side_18': 1}, {'auto_view_back_2': 3}, {'other_views_100': 38}, {'auto_view_3_4_back_0': 1}, {'other_views_100': 4}, {'auto_view_front_7': 3}, {'other_views_100': 1}, {'auto_view_front_7': 17}, {'auto_view_back_2': 1}, {'auto_view_front_7': 15}, {'auto_view_3_4_back_0': 1}, {'auto_view_front_7': 1}, {'other_views_100': 1}, {'auto_view_front_7': 1}, {'auto_view_3_4_back_0': 1}, {'auto_view_front_7': 2}, {'auto_view_3_4_back_0': 1}, {'other_views_100': 14}, {'auto_view_3_4_front_1': 1}, {'other_views_100': 15}, {'auto_view_3_4_front_1': 1}, {'other_views_100': 4}, {'auto_view_3_4_front_1': 3}, {'other_views_100': 13}, {'auto_view_side_18': 1}, {'other_views_100': 1}, {'auto_view_side_18': 8}, {'auto_view_3_4_front_1': 1}, {'auto_view_side_18': 6}, {'auto_view_3_4_front_1': 3}, {'auto_view_side_18': 2}, {'other_views_100': 1}, {'auto_view_3_4_front_1': 1}, {'auto_view_side_18': 3}, {'auto_view_fr..."
248,completed,UNKNOWN,"{'auto_view_back_2': 29, 'other_views_100': 3, 'auto_view_front_7': 12, 'auto_view_3_4_front_1': 3, 'auto_view_side_18': 43, 'auto_view_3_4_back_0': 13}",images_from_video/videos_ok_fde_d84_fded84f8-d1d4-4f2d-974b-b7637c86dfa7,"[{'auto_view_back_2': 9}, {'other_views_100': 2}, {'auto_view_front_7': 6}, {'auto_view_3_4_front_1': 1}, {'auto_view_side_18': 2}, {'auto_view_3_4_front_1': 1}, {'auto_view_side_18': 20}, {'auto_view_3_4_back_0': 5}, {'auto_view_back_2': 15}, {'auto_view_3_4_back_0': 8}, {'auto_view_side_18': 21}, {'auto_view_3_4_front_1': 1}, {'auto_view_front_7': 6}, {'auto_view_back_2': 3}, {'other_views_100': 1}, {'auto_view_back_2': 2}]"
257,completed,GOOD,"{'auto_view_side_18': 40, 'auto_view_3_4_front_1': 33, 'auto_view_3_4_back_0': 12, 'auto_view_front_7': 68, 'auto_view_back_2': 10, 'other_views_100': 2}",images_from_video/videos_ok_ad8_21a_ad821a31-f3cf-45d2-9ae3-94722aea5cda,"[{'auto_view_side_18': 1}, {'auto_view_3_4_front_1': 1}, {'auto_view_side_18': 9}, {'auto_view_3_4_back_0': 3}, {'auto_view_side_18': 6}, {'auto_view_front_7': 15}, {'auto_view_3_4_back_0': 2}, {'auto_view_front_7': 1}, {'auto_view_3_4_back_0': 1}, {'auto_view_front_7': 1}, {'auto_view_3_4_back_0': 2}, {'auto_view_front_7': 4}, {'auto_view_back_2': 3}, {'auto_view_front_7': 1}, {'auto_view_back_2': 7}, {'auto_view_front_7': 11}, {'auto_view_3_4_back_0': 1}, {'auto_view_front_7': 1}, {'auto_view_3_4_back_0': 3}, {'auto_view_front_7': 8}, {'auto_view_side_18': 1}, {'auto_view_front_7': 2}, {'auto_view_side_18': 14}, {'auto_view_3_4_front_1': 2}, {'auto_view_side_18': 1}, {'auto_view_3_4_front_1': 1}, {'auto_view_side_18': 1}, {'auto_view_3_4_front_1': 14}, {'auto_view_front_7': 23}, {'auto_view_3_4_front_1': 13}, {'other_views_100': 2}, {'auto_view_3_4_front_1': 1}, {'auto_view_side_18': 2},

In [706]:
path = 'images_from_video/videos_ok_d1f_b8a_d1fb8aee-c342-488c-8c6d-318c9017c047'
# 'other_views_100': 109 - Honde

path = 'images_from_video/videos_ok_561_615_561615c5-4564-4af6-a6b4-4cb0bc26900f'
# VW in the dark
# 'other_views_100': 38,

path = 'images_from_video/videos_ok_7a6_3e4_7a63e421-742f-43b0-ab9a-6e6909bb4764'
# small number of frames / BAD status



# path = path + '/1.jpg'
path = path + '/300.jpg'
# iimage(path)


In [238]:
pd.crosstab(df_final.classifier_result, df_final.all_racurses_present)

all_racurses_present,False,True
classifier_result,,
BAD,10,48
GOOD,29,212
UNKNOWN,2,51


In [239]:
pd.crosstab(df_final.panorama_status, df_final.all_racurses_present)

all_racurses_present,False,True
panorama_status,,
completed,22,224
failed,19,87


#### Analyze videos where not all frames are presented

In [251]:
df_failed_racurses = df_final[df_final.all_racurses_present == False]

In [303]:
len(df_failed_racurses)

41

In [304]:
len(df_res_racurses)

352

In [252]:
sr = df_failed_racurses.sample(30)

In [707]:
# sr[cols]

In [411]:
def debug_folder(path, rate = 36):
    all_files = glob.glob(path + "/*.jpg")
    all_files.sort(key=cmp_to_key(sort_by_filenumber))
    i = 0
    images = []
    for file in all_files:
        # process 2 frames per second
        i += 1
        if i%rate == 1 or i == 1 or i == len(all_files):
            images.append(file)
            
    display(*map(iimage, images))

In [708]:
path = 'images_from_video/videos_ok_991_b84_991b84f5-8f05-4078-94a5-9e35b09276bc'
# no 3/4 back - mercedez jeep

path = 'images_from_video/videos_ok_ec9_69c_ec969c78-1dd8-4680-aa81-013c2377c303'
# no back - mercedez GLC

path = 'images_from_video/videos_failed_cba_ac0_cbaac0d9-68f3-490c-ace9-673e0ec38f01'
# truck mercedez - all ok, classifier wrong - no 3-4 front

path = 'images_from_video/videos_ok_6ed_2eb_6ed2eb6a-fe9a-4c5f-a3d0-627f76c03c68'
# no front - китайский паркетник

path = 'images_from_video/videos_ok_e07_0c5_e070c5ff-09b3-4715-b6b5-fdbc60970ba3'
# no 3/4 front - KAMAZ

path = 'images_from_video/videos_ok_741_06b_74106bb5-450d-4c75-9ae2-35acd0ebee61'
# no 3/4 back - mercedez jeep

path = 'images_from_video/videos_failed_d09_b98_d09b9863-4a2d-498e-a999-a74aec917a8c'
# notebook trash

path = 'images_from_video/videos_failed_4f0_3ed_4f03edd2-5ab8-487a-8cce-eb98eb92cb37'
# not 3/4 front SCANIA truck

path = 'images_from_video/videos_ok_d48_130_d48130d1-db2b-41e7-92bb-d7b333030b1a'
# no 3/4 back - suzuki универсал

path = 'images_from_video/videos_failed_07d_373_07d37364-2f84-486a-a3ea-0dc96873edad'
# very strange part of the truck - no front

path = 'images_from_video/videos_ok_ed7_34f_ed734fea-125c-4d51-8698-6881e5e0f5ac'
# not 3/4 back opel sedan

path = 'images_from_video/videos_failed_e12_8a5_e128a5ad-9ecf-4056-9103-ea6e5cbf27b8'
# not 3/4 back truck MAN

path = 'images_from_video/videos_ok_5c0_8d9_5c08d930-2325-493b-8279-cbd6f8a3db40'
# no 3/4 front: only back part of the truck

path = 'images_from_video/videos_ok_08c_9cf_08c9cf37-68eb-474d-a5ac-3c9104a1b7a9'
# no back: mercedez jeep

path = 'images_from_video/videos_failed_986_879_986879c4-584d-44d6-89e5-1aa0f37a5983'
# no 3/4 front: Truck MAN

path = 'images_from_video/videos_failed_0c4_dea_0c4deac6-8622-4434-9919-ba84b6858925'
# no 3/4 front: truck man

path = 'images_from_video/videos_failed_7f3_b33_7f3b33f1-ffa5-49a3-9dbc-04edb3a16a1f'
# no 3/4 front: truck MAN

path = 'images_from_video/videos_ok_5c8_a96_5c8a9690-0456-4b54-8c18-a26e706f5566'
# not 3/4 front: truck SCANIA

path = 'images_from_video/videos_ok_5bf_9e0_5bf9e06d-0df8-459d-95d9-82b0c94a1114'
# no back:  китайский универсал

path = 'images_from_video/videos_ok_461_7b3_4617b3f5-1de7-4cb4-a96e-51c038c51977'
# no 3/4 front: truck Mercedez

path = 'images_from_video/videos_failed_430_9fd_4309fd5f-d7f2-4ee5-b256-5ddb6c262648'
# no back: mercedez jeep

path = 'images_from_video/videos_ok_67b_f43_67bf4320-eb00-4c00-9729-e65d1036edd2'
# truck scania: no 3/4 front, no 3/4 back

path = 'images_from_video/videos_ok_e4a_ba3_e4aba3a6-46a8-46c3-be30-12fc9a5297e0'
# no 3/4 back: jeep honda

path = 'images_from_video/videos_failed_436_d36_436d36aa-393d-4881-9fe9-e80016d38232'
# no 3/4 back: truck man

path = 'images_from_video/videos_ok_481_f9c_481f9c9f-9f26-4464-a9b5-8adb15dcfa37'
# no back: mercedez jeep

path = 'images_from_video/videos_failed_c96_89e_c9689eec-9017-4036-9e84-ae430318a5b0'
# no 3/4 back: car on the table (too many other views)

path = 'images_from_video/videos_ok_c42_759_c42759ac-4250-4a1f-9816-5f4876eec408'
# no front: renault logan

path = 'images_from_video/videos_ok_485_00e_48500e30-3e82-4a54-876e-e133a787fc59'
# no 3/4 front: обрезанный местами грузовик

path = 'images_from_video/videos_ok_082_ced_082ced9f-1d22-466d-8e2d-72c6e6da637c'
# no 3/4 front: размытый гелик

path = 'images_from_video/videos_failed_b5c_7f9_b5c7f9a6-0d70-4d57-af95-4a84ce05bacc'
# no back mercedez jeep

# img_path = path + '/600.jpg'
# iimage(img_path)
# debug_folder(path, rate = 60)




In [390]:
df_final[df_final.image_path == 'images_from_video/videos_ok_5c0_8d9_5c08d930-2325-493b-8279-cbd6f8a3db40']

,url,panorama_status,image_path,classifier_result,total_analyzed_frames,big_vehicle_middle_screen_found,no_big_vehicle_middle_screen_found,car_found_but_bounding_box_unsafe_margin,bounding_boxes_outside_image,bad_frames,really_bad_frames,share_no_big_vehicle_middle_screen_found,share_car_found_but_bounding_box_unsafe_margin,share_bad_frames,share_really_bad_frames,racurses_dict,list_racurses,all_racurses_present,classifier_racurses
71,https://autoru-panorama-internal.s3.mds.yandex.net/upload/5c0/8d9/5c08d930-2325-493b-8279-cbd6f8a3db40,completed,images_from_video/videos_ok_5c0_8d9_5c08d930-2325-493b-8279-cbd6f8a3db40,GOOD,266,262,0,0,0,4,4,0.015038,0.0,0.015038,0.015038,"{'auto_view_front_7': 39, 'auto_view_3_4_front_1': 23, 'auto_view_back_2': 44, 'auto_view_side_18': 160}","[{'auto_view_front_7': 14}, {'auto_view_3_4_front_1': 1}, {'auto_view_front_7': 1}, {'auto_view_3_4_front_1': 2}, {'auto_view_back_2': 1}, {'auto_view_3_4_front_1': 4}, {'auto_view_side_18': 40}, {'auto_view_front_7': 1}, {'auto_view_side_18': 4}, {'auto_view_back_2': 1}, {'auto_view_front_7': 2}, {'auto_view_side_18': 21}, {'auto_view_back_2': 1}, {'auto_view_side_18': 4}, {'auto_view_back_2': 2}, {'auto_view_side_18': 3}, {'auto_view_back_2': 3}, {'auto_view_side_18': 2}, {'auto_view_back_2': 1}, {'auto_view_side_18': 3}, {'auto_view_back_2': 1}, {'auto_view_3_4_front_1': 1}, {'auto_view_back_2': 1}, {'auto_view_3_4_front_1': 1}, {'auto_view_back_2': 28}, {'auto_view_side_18': 5}, {'auto_view_back_2': 1}, {'auto_view_side_18': 33}, {'auto_view_back_2': 1}, {'auto_view_side_18': 24}, {'auto_view_back_2': 2}, {'auto_view_side_18': 2}, {'auto_view_back_2': 1}, {'auto_view_side_18': 14}, {'auto_view_front_7': 1}, {'auto_view_side_18': 1}, {'auto_view_3_4_front_1': 1}, {'auto_view_sid...",False,UNKNOWN_RACURS


### Classify racurses

In [359]:
AUTO_VIEW_3_4_BACK = "auto_view_3_4_back_0"
AUTO_VIEW_3_4_FRONT = "auto_view_3_4_front_1"
AUTO_VIEW_BACK = "auto_view_back_2"
AUTO_VIEW_FRONT = "auto_view_front_7"
AUTO_VIEW_SIDE = "auto_view_side_18"
AUTO_VIEW_OTHER = "other_views_100"

In [715]:
min_share_side = 0.15 # 1/2-1/4
min_share_back_front = 0.10
max_other_share = 0.05
min_share_3_4_front_back = 0.10
min_total_share_back_front = 0.20
GOOD_RACURS = 'GOOD_RACURS'
OK_RACURS = 'OK_RACURS'
BAD_RACURS = 'BAD_RACURS'
UNKNOWN_RACURS = 'UNKNOWN_RACURS'
ZERO_FRAMES_RACURS = 'ZERO_FRAMES_RACURS'
def classify_racurses(r_dict):
    total_frames = sum(r_dict.values())
    if total_frames == 0:
        return ZERO_FRAMES_RACURS
        
    side_share = r_dict.get(AUTO_VIEW_SIDE, 0)/total_frames
    back_share = r_dict.get(AUTO_VIEW_BACK, 0)/total_frames
    front_share = r_dict.get(AUTO_VIEW_FRONT, 0)/total_frames
    back_3_4_share = r_dict.get(AUTO_VIEW_3_4_BACK, 0)/total_frames
    front_3_4_share = r_dict.get(AUTO_VIEW_3_4_FRONT, 0)/total_frames
    back_total_share = back_3_4_share + back_share
    front_total_share = front_3_4_share + front_share
    other_share = r_dict.get(AUTO_VIEW_OTHER, 0)/total_frames

    # everything is great: enough share for all racurses and small amount of bad photos
    if side_share >= min_share_side \
        and front_share >= min_share_back_front \
        and back_share >= min_share_back_front \
        and front_3_4_share >= min_share_3_4_front_back \
        and back_3_4_share >= min_share_3_4_front_back \
        and other_share <= max_other_share:
        return GOOD_RACURS

    # for trucks and jeeps 3/4 back is mixed with back in current model, and 3/4 front is mixed with front
    if side_share >= min_share_side \
        and front_total_share >= min_total_share_back_front \
        and back_total_share >= min_total_share_back_front \
        and other_share <= max_other_share:
        return OK_RACURS
    
    if side_share == 0 or back_total_share == 0 or front_total_share == 0 or other_share > 0.5:
        return BAD_RACURS
    else:
        return UNKNOWN_RACURS
    
        

    

In [378]:
df_final['classifier_racurses'] = df_final.racurses_dict.map(classify_racurses)

In [379]:
df_final.classifier_racurses.value_counts()

UNKNOWN_RACURS        134
GOOD_RACURS           117
OK_RACURS              93
BAD_RACURS              6
ZERO_FRAMES_RACURS      2
Name: classifier_racurses, dtype: int64

In [380]:
df_final[df_final.classifier_racurses == BAD_RACURS][cols]

,panorama_status,classifier_result,racurses_dict,image_path,list_racurses
95,failed,BAD,{'other_views_100': 94},images_from_video/videos_failed_d09_b98_d09b9863-4a2d-498e-a999-a74aec917a8c,[{'other_views_100': 94}]
114,completed,UNKNOWN,"{'auto_view_3_4_back_0': 38, 'auto_view_side_18': 73, 'auto_view_back_2': 28}",images_from_video/videos_ok_59e_aa7_59eaa7fc-c014-40de-a81d-fbb2d83570dc,"[{'auto_view_3_4_back_0': 10}, {'auto_view_side_18': 31}, {'auto_view_3_4_back_0': 3}, {'auto_view_side_18': 11}, {'auto_view_back_2': 1}, {'auto_view_side_18': 1}, {'auto_view_back_2': 23}, {'auto_view_3_4_back_0': 6}, {'auto_view_side_18': 1}, {'auto_view_3_4_back_0': 1}, {'auto_view_side_18': 6}, {'auto_view_3_4_back_0': 1}, {'auto_view_side_18': 2}, {'auto_view_3_4_back_0': 1}, {'auto_view_side_18': 11}, {'auto_view_3_4_back_0': 1}, {'auto_view_side_18': 1}, {'auto_view_3_4_back_0': 4}, {'auto_view_side_18': 1}, {'auto_view_3_4_back_0': 11}, {'auto_view_back_2': 1}, {'auto_view_side_18': 1}, {'auto_view_back_2': 3}, {'auto_view_side_18': 7}]"
169,failed,BAD,"{'auto_view_3_4_back_0': 5, 'auto_view_3_4_front_1': 5, 'auto_view_front_7': 11, 'auto_view_side_18': 13, 'other_views_100': 55, 'auto_view_back_2': 15}",images_from_video/videos_failed_d0b_0b4_d0b0b4eb-ed26-460d-a360-d00263f94b1d,"[{'auto_view_3_4_back_0': 3}, {'auto_view_3_4_front_1': 1}, {'auto_view_3_4_back_0': 1}, {'auto_view_3_4_front_1': 3}, {'auto_view_front_7': 9}, {'auto_view_side_18': 2}, {'other_views_100': 20}, {'auto_view_back_2': 8}, {'auto_view_front_7': 1}, {'auto_view_back_2': 5}, {'auto_view_front_7': 1}, {'auto_view_back_2': 2}, {'auto_view_3_4_back_0': 1}, {'other_views_100': 1}, {'auto_view_3_4_front_1': 1}, {'other_views_100': 6}, {'auto_view_side_18': 1}, {'other_views_100': 8}, {'auto_view_side_18': 1}, {'other_views_100': 20}, {'auto_view_side_18': 9}]"
215,failed,BAD,"{'other_views_100': 85, 'auto_view_front_7': 55}",images_from_video/videos_failed_665_30c_66530cad-5a2b-4823-8592-df28c5cb306c,"[{'other_views_100': 15}, {'auto_view_front_7': 6}, {'other_views_100': 6}, {'auto_view_front_7': 1}, {'other_views_100': 1}, {'auto_view_front_7': 1}, {'other_views_100': 6}, {'auto_view_front_7': 2}, {'other_views_100': 4}, {'auto_view_front_7': 1}, {'other_views_100': 4}, {'auto_view_front_7': 1}, {'other_views_100': 1}, {'auto_view_front_7': 13}, {'other_views_100': 1}, {'auto_view_front_7': 5}, {'other_views_100': 1}, {'auto_view_front_7': 2}, {'other_views_100': 12}, {'auto_view_front_7': 1}, {'other_views_100': 1}, {'auto_view_front_7': 2}, {'other_views_100': 5}, {'auto_view_front_7': 12}, {'other_views_100': 2}, {'auto_view_front_7': 8}, {'other_views_100': 26}]"
220,failed,BAD,{'other_views_100': 81},images_from_video/videos_failed_596_adb_596adbd6-3fba-407c-a2aa-da1cd9649d09,[{'other_views_100': 81}]
338,failed,BAD,"{'other_views_100': 178, 'auto_view_3_4_back_0': 3}",images_from_video/videos_failed_826_f70_826f70cb-f5ec-4a5b-bcc1-d7d436f27ea9,"[{'other_views_100': 61}, {'auto_view_3_4_back_0': 1}, {'other_views_100': 2}, {'auto_view_3_4_back_0': 1}, {'other_views_100': 21}, {'auto_view_3_4_back_0': 1}, {'other_views_100': 94}]"


In [709]:
# path = 'images_from_video/videos_failed_ff9_fd6_ff9fd619-927c-4cab-ae02-a0bcc4665ab4'
# обрезанное видео

# path = 'images_from_video/videos_ok_59e_aa7_59eaa7fc-c014-40de-a81d-fbb2d83570dc'
# truck isuzu no fron

path = 'images_from_video/videos_ok_d1f_b8a_d1fb8aee-c342-488c-8c6d-318c9017c047'
# kia - много other кадров в поле

path = 'images_from_video/videos_ok_561_615_561615c5-4564-4af6-a6b4-4cb0bc26900f'
# toyota в темноте - много other кадров

path = 'images_from_video/videos_ok_828_edc_828edc3a-e6f5-44ef-95e1-2dcdc008d89a'
# снова kia

# debug_folder(path, rate = 60)

In [414]:
GOOD = 'GOOD'
UNKNOWN = 'UNKNOWN'
good_bb = df_final[df_final.classifier_result == GOOD]

In [415]:
unknown_bb = df_final[df_final.classifier_result == UNKNOWN]
df_final.classifier_result.value_counts()

GOOD       241
BAD         58
UNKNOWN     53
Name: classifier_result, dtype: int64

In [384]:
len(good_bb)

241

In [386]:
good_bb.classifier_racurses.value_counts()

UNKNOWN_RACURS    92
GOOD_RACURS       85
OK_RACURS         64
Name: classifier_racurses, dtype: int64

In [417]:
unknown_bb.classifier_racurses.value_counts()

UNKNOWN_RACURS    25
GOOD_RACURS       16
OK_RACURS         11
BAD_RACURS         1
Name: classifier_racurses, dtype: int64

In [388]:
pd.crosstab(good_bb.classifier_racurses, good_bb.panorama_status)

panorama_status,completed,failed
classifier_racurses,,
GOOD_RACURS,71,14
OK_RACURS,50,14
UNKNOWN_RACURS,64,28


In [418]:
pd.crosstab(unknown_bb.classifier_racurses, unknown_bb.panorama_status)

panorama_status,completed,failed
classifier_racurses,,
BAD_RACURS,1,0
GOOD_RACURS,12,4
OK_RACURS,9,2
UNKNOWN_RACURS,12,13


In [389]:
good_bb_failed = good_bb[good_bb.panorama_status == 'failed']

In [431]:
unknown_bb_failed = unknown_bb[unknown_bb.panorama_status == 'failed']

In [432]:
good_racurs_failed = good_bb_failed[good_bb_failed.classifier_racurses == GOOD_RACURS]
unk_good_racurs_failed = unknown_bb_failed[unknown_bb_failed.classifier_racurses == GOOD_RACURS]

In [433]:
ok_racurs_failed = good_bb_failed[good_bb_failed.classifier_racurses == OK_RACURS]
unk_ok_racurs_failed = unknown_bb_failed[unknown_bb_failed.classifier_racurses == OK_RACURS]

In [434]:
unknown_racurs_failed = good_bb_failed[good_bb_failed.classifier_racurses == UNKNOWN_RACURS]
unk_unknown_racurs_failed = unknown_bb_failed[unknown_bb_failed.classifier_racurses == UNKNOWN_RACURS]

In [435]:
len(good_racurs_failed)

14

In [407]:
len(ok_racurs_failed)

14

In [404]:
def debug_folders(df, rate = 60):
    paths = list(df.image_path)
    for path in paths:
        print(path)
        debug_folder(path, rate = rate)

In [444]:
# debug_folders(good_racurs_failed, rate = 120)
# all ok

# debug_folders(ok_racurs_failed, rate = 120)
# all ok

# debug_folders(unknown_racurs_failed, rate = 120)
# недокрутил 2 машины:
# images_from_video/videos_failed_250_88e_25088e17-e8fd-4f12-9206-00a3853e03f6
# тут еще и логотип

# images_from_video/videos_failed_8b2_5cf_8b25cf8c-c03c-4c3b-86bf-b47863f75b32
# Недокруитил? 

# print(len(unk_good_racurs_failed))
# debug_folders(unk_good_racurs_failed)
# images_from_video/videos_failed_9fc_caa_9fccaad4-d893-4e51-ab31-0b13af9c6256 недокрутил
# почти докрутил: images_from_video/videos_failed_786_168_786168b6-6d12-4f1d-a299-b53bdab2faa8
# та же, недокрутил: images_from_video/videos_failed_03f_4a8_03f4a857-3291-48b3-8bcd-7b3103f08a7e
# застрял на последнем кадре

# print(len(unk_ok_racurs_failed))
# debug_folders(unk_ok_racurs_failed, rate = 120)
# images_from_video/videos_failed_b53_c69_b53c69b5-33d6-4748-95e1-f842982f727c - застрял в последних кадрах
# images_from_video/videos_failed_af9_ee7_af9ee745-2571-414e-ae9f-5cde82e9dfda - недокрутил
# images_from_video/videos_failed_ebb_b2f_ebbb2f60-3561-4fb8-a279-cb22550d743d - перекрутил немного
# images_from_video/videos_failed_14d_1cd_14d1cded-1861-44c3-831c-5edcc753c20b - недокрутил

# print(len(unk_unknown_racurs_failed))
# debug_folders(unk_unknown_racurs_failed, rate = 120)


#### Недокрученное видео

In [553]:
# RACURSES_INDEXES = {
#     "auto_view_3_4_back_0": 0
#     "auto_view_back_2"
    
#                            '1': "auto_view_3_4_front_1",
#                            '2': "auto_view_back_2",
#                            '7': "auto_view_front_7",
#                            '18': "auto_view_side_18",
#                            "100": "other_views_100"}
    
# }
CAN_FOLLOW_STRICT = 'CAN_FOLLOW_STRICT'
CAN_FOLLOW_LIGHT= 'CAN_FOLLOW_LIGHT'
CANNOT_FOLLOW = 'CANNOT_FOLLOW'
def can_follow(racurs1, racurs2):
    if racurs1 == AUTO_VIEW_3_4_BACK:
        if racurs2 in {AUTO_VIEW_BACK, AUTO_VIEW_SIDE}:
            return CAN_FOLLOW_STRICT
        elif racurs2 in {AUTO_VIEW_3_4_FRONT}: 
            return CAN_FOLLOW_LIGHT
        else:
            return CANNOT_FOLLOW
    if racurs1 == AUTO_VIEW_SIDE:
        if racurs2 in (AUTO_VIEW_3_4_BACK, AUTO_VIEW_3_4_FRONT):
            return CAN_FOLLOW_STRICT
        elif racurs2 in {AUTO_VIEW_FRONT, AUTO_VIEW_BACK}:
            return CAN_FOLLOW_LIGHT
        else:
            return CANNOT_FOLLOW
        
    if racurs1 == AUTO_VIEW_3_4_FRONT:
        if racurs2 in {AUTO_VIEW_FRONT, AUTO_VIEW_SIDE}:
            return CAN_FOLLOW_STRICT
        elif racurs2 in {AUTO_VIEW_3_4_BACK}: 
            return CAN_FOLLOW_LIGHT
        else:
            return CANNOT_FOLLOW
    if racurs1 == AUTO_VIEW_BACK:
        if racurs2 in {AUTO_VIEW_3_4_BACK}:
            return CAN_FOLLOW_STRICT
        elif racurs2 in {AUTO_VIEW_SIDE}: 
            return CAN_FOLLOW_LIGHT
        else:
            return CANNOT_FOLLOW
    if racurs1 == AUTO_VIEW_FRONT:
        if racurs2 in {AUTO_VIEW_3_4_FRONT}:
            return CAN_FOLLOW_STRICT
        elif racurs2 in {AUTO_VIEW_SIDE}: 
            return CAN_FOLLOW_LIGHT
        else:
            return CANNOT_FOLLOW

In [717]:
RACURS_2_MIN_COUNT = {
    AUTO_VIEW_3_4_BACK: 2,
    AUTO_VIEW_3_4_FRONT: 2,
    AUTO_VIEW_SIDE: 2,
    AUTO_VIEW_BACK: 1,
    AUTO_VIEW_FRONT: 1
}
def is_finished(first_racurs, racurses_2_count):
    if racurses_2_count[first_racurs] < RACURS_2_MIN_COUNT[first_racurs] + 1:
        return False
    for key, value in RACURS_2_MIN_COUNT.items():
        if value > racurses_2_count.get(key, 0):
            return False
    return True
    
    
def classify_video_finished_by_row(row):
    return classify_video_finished(row['list_racurses'])

def classify_video_finished(list_racurses):
    
    list_ok_racurses = []
    
    for racurs in list_racurses:
#         print(racurs)
        key, value = list(racurs.items())[0]
        if value <= 2 or key == AUTO_VIEW_OTHER:
            continue
        else:
            # join only same racurses:
            if not ((len(list_ok_racurses) > 0) and (key == list_ok_racurses[-1])):
                list_ok_racurses.append(key)

    if len(list_ok_racurses) < 4:
        return CANNOT_FOLLOW, False


    current_can_follow = CAN_FOLLOW_STRICT
    racurses2count = defaultdict(int)

    
    for i in range(len(list_ok_racurses) - 1):
        racurs1 = list_ok_racurses[i]
        racurs2 = list_ok_racurses[i+1]
        if current_can_follow != CANNOT_FOLLOW:
            can_follow_res = can_follow(racurs1, racurs2)
            if can_follow_res == CANNOT_FOLLOW:
                current_can_follow = can_follow_res
            elif can_follow_res == CAN_FOLLOW_LIGHT:
                current_can_follow = can_follow_res
        racurses2count[racurs1] += 1
    racurses2count[list_ok_racurses[-1]] += 1
    is_finished_result = is_finished(list_ok_racurses[0], racurses2count)
    return current_can_follow, is_finished_result


In [671]:
df_final[['can_follow_racurses', 'is_video_finished']] = df_final.apply(classify_video_finished_by_row, axis = 1, result_type='expand')

In [672]:
df_follow_strict = df_final[(df_final.can_follow_racurses == CAN_FOLLOW_STRICT)]

In [674]:
df_final.can_follow_racurses.value_counts()

CANNOT_FOLLOW        160
CAN_FOLLOW_STRICT    101
CAN_FOLLOW_LIGHT      91
Name: can_follow_racurses, dtype: int64

In [675]:
pd.crosstab(df_final.classifier_racurses, df_final.can_follow_racurses)

can_follow_racurses,CANNOT_FOLLOW,CAN_FOLLOW_LIGHT,CAN_FOLLOW_STRICT
classifier_racurses,,,
BAD_RACURS,5,1,0
GOOD_RACURS,13,25,79
OK_RACURS,57,17,19
UNKNOWN_RACURS,83,48,3
ZERO_FRAMES_RACURS,2,0,0


In [676]:
pd.crosstab(df_final.classifier_result, df_final.can_follow_racurses)

can_follow_racurses,CANNOT_FOLLOW,CAN_FOLLOW_LIGHT,CAN_FOLLOW_STRICT
classifier_result,,,
BAD,27,10,21
GOOD,111,60,70
UNKNOWN,22,21,10


In [678]:
good_bb = df_final[df_final.classifier_result == GOOD]

In [679]:
pd.crosstab(good_bb.panorama_status, good_bb.can_follow_racurses)

can_follow_racurses,CANNOT_FOLLOW,CAN_FOLLOW_LIGHT,CAN_FOLLOW_STRICT
panorama_status,,,
completed,77,50,58
failed,34,10,12


In [681]:
can_follow_ok = df_final[df_final.classifier_result.isin([GOOD, UNKNOWN]) & (df_final.can_follow_racurses == CAN_FOLLOW_STRICT)]

In [683]:
pd.crosstab(can_follow_ok.panorama_status, can_follow_ok.is_video_finished)

is_video_finished,False,True
panorama_status,,
completed,6,59
failed,3,12


In [ ]:
pd.crosstab(good_bb.panorama_status, good_bb.can_follow_racurses)

In [673]:
df_follow_strict['is_video_finished'].value_counts()

True     80
False    21
Name: is_video_finished, dtype: int64

### failed panoramas not finished analysis


In [686]:
failed_finished = list(can_follow_ok[(can_follow_ok.panorama_status == 'failed') &  (can_follow_ok.is_video_finished == True)].image_path)

In [690]:
len(failed_finished)

12

In [691]:
# failed_finished
# debug_folders(df_final[df_final.image_path.isin(failed_finished)], rate = 120)
# # all ok, really finished

In [692]:
failed_unfinished = list(can_follow_ok[(can_follow_ok.panorama_status == 'failed') &  (can_follow_ok.is_video_finished == False)].image_path)

In [693]:
len(failed_unfinished)

3

In [695]:
failed_unfinished
# all true

['images_from_video/videos_failed_250_88e_25088e17-e8fd-4f12-9206-00a3853e03f6',
 'images_from_video/videos_failed_9fc_caa_9fccaad4-d893-4e51-ab31-0b13af9c6256',
 'images_from_video/videos_failed_03f_4a8_03f4a857-3291-48b3-8bcd-7b3103f08a7e']

In [703]:
for k in list(df_final[df_final.image_path.isin(failed_unfinished)].url):
    print(k)

https://autoru-panorama-internal.s3.mds.yandex.net/upload/250/88e/25088e17-e8fd-4f12-9206-00a3853e03f6
https://autoru-panorama-internal.s3.mds.yandex.net/upload/9fc/caa/9fccaad4-d893-4e51-ab31-0b13af9c6256
https://autoru-panorama-internal.s3.mds.yandex.net/upload/03f/4a8/03f4a857-3291-48b3-8bcd-7b3103f08a7e


In [696]:
# debug_folders(df_final[df_final.image_path.isin(failed_unfinished)], rate = 120)
# all unfinished

In [697]:
completed_unfinished = list(can_follow_ok[(can_follow_ok.panorama_status == 'completed') &  (can_follow_ok.is_video_finished == False)].image_path)

In [704]:
completed_unfinished_razmetka = [
    'images_from_video/videos_ok_47f_713_47f71340-3f3c-48e1-9f8e-461d9270614c',
    'images_from_video/videos_ok_e03_8dc_e038dc0a-2e34-4325-89cb-1b616f3e8b62',
    'images_from_video/videos_ok_8de_69c_8de69c13-a9b7-467b-a449-c1ee452ccd90',
    'images_from_video/videos_ok_87a_f2d_87af2d62-f206-4ab2-8550-23f82f27b8e1'
]
for k in list(df_final[df_final.image_path.isin(completed_unfinished_razmetka)].url):
    print(k)

https://autoru-panorama-internal.s3.mds.yandex.net/upload/47f/713/47f71340-3f3c-48e1-9f8e-461d9270614c
https://autoru-panorama-internal.s3.mds.yandex.net/upload/e03/8dc/e038dc0a-2e34-4325-89cb-1b616f3e8b62
https://autoru-panorama-internal.s3.mds.yandex.net/upload/8de/69c/8de69c13-a9b7-467b-a449-c1ee452ccd90
https://autoru-panorama-internal.s3.mds.yandex.net/upload/87a/f2d/87af2d62-f206-4ab2-8550-23f82f27b8e1


In [698]:
completed_unfinished
# 2 докрутили, 4 недокрутили
# images_from_video/videos_ok_47f_713_47f71340-3f3c-48e1-9f8e-461d9270614c
# images_from_video/videos_ok_e03_8dc_e038dc0a-2e34-4325-89cb-1b616f3e8b62
# 'images_from_video/videos_ok_8de_69c_8de69c13-a9b7-467b-a449-c1ee452ccd90'
# 'images_from_video/videos_ok_87a_f2d_87af2d62-f206-4ab2-8550-23f82f27b8e1'

['images_from_video/videos_ok_53b_d70_53bd70c9-a263-459d-8fc6-f2279576b965',
 'images_from_video/videos_ok_591_830_59183004-1a31-4b29-9122-74b201574e76',
 'images_from_video/videos_ok_47f_713_47f71340-3f3c-48e1-9f8e-461d9270614c',
 'images_from_video/videos_ok_e03_8dc_e038dc0a-2e34-4325-89cb-1b616f3e8b62',
 'images_from_video/videos_ok_8de_69c_8de69c13-a9b7-467b-a449-c1ee452ccd90',
 'images_from_video/videos_ok_87a_f2d_87af2d62-f206-4ab2-8550-23f82f27b8e1']

In [710]:
# debug_folders(df_final[df_final.image_path.isin(completed_unfinished)], rate = 120)

In [712]:
df_final.to_csv('bb_racurs_classifier_result.tsv', sep = '\t', index = False)

In [713]:
list(df_final)

['url',
 'panorama_status',
 'image_path',
 'classifier_result',
 'total_analyzed_frames',
 'big_vehicle_middle_screen_found',
 'no_big_vehicle_middle_screen_found',
 'car_found_but_bounding_box_unsafe_margin',
 'bounding_boxes_outside_image',
 'bad_frames',
 'really_bad_frames',
 'share_no_big_vehicle_middle_screen_found',
 'share_car_found_but_bounding_box_unsafe_margin',
 'share_bad_frames',
 'share_really_bad_frames',
 'racurses_dict',
 'list_racurses',
 'all_racurses_present',
 'classifier_racurses',
 'can_follow_racurses',
 'is_video_finished']

In [714]:
df_final.all_racurses_present.value_counts()

True     311
False     41
Name: all_racurses_present, dtype: int64

In [716]:
df_final.classifier_racurses.value_counts()

UNKNOWN_RACURS        134
GOOD_RACURS           117
OK_RACURS              93
BAD_RACURS              6
ZERO_FRAMES_RACURS      2
Name: classifier_racurses, dtype: int64

In [718]:
df_final.can_follow_racurses.value_counts()

CANNOT_FOLLOW        160
CAN_FOLLOW_STRICT    101
CAN_FOLLOW_LIGHT      91
Name: can_follow_racurses, dtype: int64

In [719]:
df_final[df_final.can_follow_racurses == CAN_FOLLOW_STRICT]['is_video_finished'].value_counts()

True     80
False    21
Name: is_video_finished, dtype: int64

### analysis of unfinished videos

In [648]:
unfinished_videos = [
    'images_from_video/videos_failed_250_88e_25088e17-e8fd-4f12-9206-00a3853e03f6', # сработал классификатор CAN_FOLLOW_STRICT, False, начали с side 20, закончили с front 1 (>2 кадров у 3_4 back)
#                      'images_from_video/videos_failed_8b2_5cf_8b25cf8c-c03c-4c3b-86bf-b47863f75b32', # CANNOT_FOLLOW начали с front, закончили 3/4 front
#                      'images_from_video/videos_failed_9fc_caa_9fccaad4-d893-4e51-ab31-0b13af9c6256', # сработал классфикатор, CAN_FOLLOW_STRICT, False, начали с 3_4 front (7 > 2) а зкончили с front - очень хорошая последовательность чуть не докрутили
#                      'images_from_video/videos_failed_786_168_786168b6-6d12-4f1d-a299-b53bdab2faa8',   #  is finished false, can follow light начали с front, закончили с 3/4 front
#                      'images_from_video/videos_failed_af9_ee7_af9ee745-2571-414e-ae9f-5cde82e9dfda', # cannot follow (false) begin with 3_4 front 13, finished with side 10
#                      'images_from_video/videos_failed_ebb_b2f_ebbb2f60-3561-4fb8-a279-cb22550d743d', # cannot follow (false) начали с front, закончили с front (car with box on the roof - a lot of problems)
#                      'images_from_video/videos_failed_14d_1cd_14d1cded-1861-44c3-831c-5edcc753c20b', # cannot follow (true) начали с 3_4 back, закончили с 3_4. fron
                    ]




In [649]:
l_racurses = list(df_final[df_final.image_path.isin(unfinished_videos)].list_racurses)[0]

In [650]:
classify_video_finished(l_racurses)

['auto_view_side_18', 'auto_view_3_4_front_1', 'auto_view_front_7', 'auto_view_3_4_front_1', 'auto_view_side_18', 'auto_view_3_4_back_0']
{'auto_view_side_18': 2, 'auto_view_3_4_front_1': 2, 'auto_view_front_7': 1, 'auto_view_3_4_back_0': 1}
CAN_FOLLOW_STRICT
False


In [711]:
# debug_folders(df_final[df_final.image_path.isin(unfinished_videos)])

### Analyze pictures from Anton

In [68]:
all_files = glob.glob('racurses_images/*/*.png')

In [121]:
def process_all_racurses(path_mask = 'racurses_images/*/*.png'):
    all_files = glob.glob(path_mask)
    all_results = []
    i = 0
    for file in all_files:
        racurs, score = get_racurs_tf_lite_result(file)
        res = {
            'path': file,
            'racurs': racurs,
            'score': score
        }
        all_results.append(res)
        i+=1
#         if i > 10:
#             break
    return all_results
    
        
    

In [88]:
all_results = process_all_racurses()

In [93]:
df = pd.DataFrame(all_results)

In [90]:
df.sample(10)

,path,racurs,score
168,racurses_images/lada/00093.png,auto_view_3_4_back_0,131
205,racurses_images/iveco/00087.png,auto_view_front_7,169
258,racurses_images/iveco/00017.png,auto_view_side_18,94
0,racurses_images/RangeRoverWhiteNoPlates/00025.png,auto_view_side_18,133
125,racurses_images/lada/00047.png,auto_view_back_2,152
248,racurses_images/iveco/00091.png,auto_view_front_7,167
127,racurses_images/lada/00053.png,auto_view_back_2,192
110,racurses_images/lada/00082.png,auto_view_side_18,157
46,racurses_images/RangeRoverWhiteNoPlates/00000.png,auto_view_front_7,183
285,racurses_images/iveco/00057.png,auto_view_back_2,144


In [110]:
path = 'racurses_images/RangeRoverWhiteNoPlates/00025.png'
path = 'racurses_images/RangeRoverWhiteNoPlates/00050.png'
path = 'racurses_images/RangeRoverWhiteNoPlates/00090.png'
# iimage(path)

### Show in html

In [94]:
from IPython.display import HTML

# df = pd.DataFrame(['<img src="racurses_images/RangeRoverWhiteNoPlates/00025.png"/>', 'racurses_images/RangeRoverWhiteNoPlates/00050.png'], columns = ['Image'])

# HTML(df.to_html(escape=False))
df['Image'] = df['path'].map(lambda x: '<img src="' + x + '"/>')

In [104]:
# html_df = df.sample(30).to_html(escape=False)
html_full_df = df.to_html(escape=False)

In [106]:
list(df)

['path', 'racurs', 'score', 'Image']

In [107]:
cols = ['path', 'racurs', 'score']
df[cols].to_csv('racurs_classifier_results.tsv', sep = '\t', index = False)

In [108]:
# HTML(html_df)

In [105]:
# with open('html_result.html', 'w') as file:
#     file.write(html_df)
with open('html_full_result.html', 'w') as file:
    file.write(html_full_df)

### Analyze video from Anton

In [133]:
# !ls racurses_images/

In [134]:
# !mkdir racurses_images/merc_video

In [135]:
# !ffmpeg -i racurses_images/fcebf650-c827-4090-9283-ddee79bffdfe.mp4 -r 3 racurses_images/merc_video/image_%05d.jpg

In [122]:
all_results = process_all_racurses(path_mask='racurses_images/merc_video/*.jpg')

In [123]:
df = pd.DataFrame(all_results)
df[cols].to_csv('merc_video_classifier_results.tsv', sep = '\t', index = False)

In [124]:
df[cols].sample(10)

,path,racurs,score
40,racurses_images/merc_video/image_00191.jpg,auto_view_side_18,160
20,racurses_images/merc_video/image_00062.jpg,auto_view_back_2,139
210,racurses_images/merc_video/image_00057.jpg,auto_view_back_2,117
157,racurses_images/merc_video/image_00208.jpg,auto_view_side_18,125
261,racurses_images/merc_video/image_00111.jpg,auto_view_side_18,125
10,racurses_images/merc_video/image_00107.jpg,auto_view_side_18,120
38,racurses_images/merc_video/image_00217.jpg,auto_view_back_2,167
256,racurses_images/merc_video/image_00196.jpg,auto_view_side_18,162
183,racurses_images/merc_video/image_00024.jpg,auto_view_3_4_front_1,167
53,racurses_images/merc_video/image_00085.jpg,auto_view_3_4_front_1,107


In [130]:
df.racurs.value_counts()

auto_view_side_18        101
auto_view_3_4_front_1     83
auto_view_front_7         44
auto_view_back_2          41
auto_view_3_4_back_0       3
Name: racurs, dtype: int64

In [125]:
df['Image'] = df['path'].map(lambda x: '<img src="' + x + '"/>')

In [126]:
s_html_df = df.sample(30).to_html(escape=False)

In [132]:
# HTML(s_html_df)

In [128]:
html_full_df = df.to_html(escape=False)

In [129]:
with open('html_full_result_merc_video.html', 'w') as file:
    file.write(html_full_df)